In [1]:
import pandas as pd
import sqlite3
import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

from sqlalchemy import create_engine

In [2]:
#connection to a PostgreSQL local database
# ===========================================

# user_name_var = input('What is the username of your database? ')
host_var = input('What is the host address of your database?')
pw = input("Enter the database password: ")
# db_name = input("Enter the name of the database: ")

string_output = "postgresql://postgres"+':'+pw+'@'+host_var+":5432"+'/earthquake_db'
engine = create_engine(string_output)
connection = engine.connect()
print("Connected to database...")

Connected to database...


In [3]:
#create and connect to a sqlite3 db
conn = sqlite3.connect(r'/Users/rosaicelaroman/Desktop/portfolio_work/earthquake_modeling/Database/db/earthquake_db')

Adding 1 month of data

In [4]:
page_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"

df = pd.read_csv(page_url)
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-06-08T03:26:30.117Z,21.138300,99.915700,10.000000,4.20,mb,NaN,76.00,0.33700,0.9600,...,2022-06-08T03:45:55.040Z,"54 km S of Kēng Tung, Myanmar",earthquake,2.70,2.00,0.263000,4.0,reviewed,us,us
1,2022-06-08T03:19:05.190Z,19.183834,-155.404663,32.259998,1.87,md,36.0,175.00,NaN,0.1100,...,2022-06-08T03:22:11.780Z,"8 km ESE of Pāhala, Hawaii",earthquake,0.81,1.19,1.380000,6.0,automatic,hv,hv
2,2022-06-08T03:15:44.700Z,38.787998,-122.767502,1.960000,1.21,md,25.0,78.00,0.01400,0.0300,...,2022-06-08T03:33:16.167Z,"2km NW of The Geysers, CA",earthquake,0.21,0.36,0.050000,5.0,automatic,nc,nc
3,2022-06-08T03:08:30.310Z,34.319833,-116.842000,4.670000,1.40,ml,33.0,93.00,0.08686,0.1900,...,2022-06-08T03:19:24.760Z,"6km N of Big Bear City, CA",earthquake,0.31,1.14,0.140000,28.0,automatic,ci,ci
4,2022-06-08T03:03:10.010Z,18.010333,-66.914500,4.080000,2.02,md,3.0,227.00,NaN,0.0900,...,2022-06-08T03:22:28.580Z,"2 km N of Fuig, Puerto Rico",earthquake,1.96,3.70,0.028947,3.0,reviewed,pr,pr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8640,2022-05-09T04:32:16.310Z,17.874167,-66.876667,13.740000,2.71,md,9.0,245.00,NaN,0.1900,...,2022-05-09T04:45:08.100Z,"11 km SSE of Guánica, Puerto Rico",earthquake,1.04,0.49,0.239170,7.0,reviewed,pr,pr
8641,2022-05-09T04:27:04.998Z,59.922600,-152.162900,79.900000,1.50,ml,NaN,NaN,NaN,0.5300,...,2022-05-09T04:30:31.925Z,"24 km W of Happy Valley, Alaska",earthquake,NaN,0.70,NaN,NaN,automatic,ak,ak
8642,2022-05-09T04:13:49.615Z,37.202100,-114.879800,5.700000,1.60,ml,5.0,231.36,0.24600,0.1384,...,2022-05-09T20:14:59.778Z,"31 km SE of Alamo, Nevada",earthquake,NaN,14.00,0.160000,2.0,reviewed,nn,nn
8643,2022-05-09T04:04:52.770Z,46.805167,-121.920667,7.390000,-0.08,md,13.0,156.00,0.04138,0.0800,...,2022-05-09T16:39:04.370Z,"9 km ENE of Ashford, Washington",earthquake,0.32,0.46,0.218354,13.0,reviewed,uw,uw


In [5]:
#append the new data from USGS to both databases
#=============
#PostgreSQL
df.to_sql('earthquake_raw', engine, if_exists='append', index = False)
#=============
#SQLite
cur = conn.cursor()
df.to_sql('earthquake_raw', conn, if_exists='append', index=False)

8645

Load from database to update with new rows and remove duplicate rows

In [6]:
#Dataframe from postgresql
model_df = pd.read_sql_table(
    "earthquake_raw",
    con=engine
)
model_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.87500,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290000,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.11900,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460000,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62577,2022-05-09T04:32:16.310Z,17.874167,-66.876667,13.74,2.71,md,9.0,245.00,NaN,0.1900,...,2022-05-09T04:45:08.100Z,"11 km SSE of Guánica, Puerto Rico",earthquake,1.04,0.49,0.239170,7.0,reviewed,pr,pr
62578,2022-05-09T04:27:04.998Z,59.922600,-152.162900,79.90,1.50,ml,NaN,NaN,NaN,0.5300,...,2022-05-09T04:30:31.925Z,"24 km W of Happy Valley, Alaska",earthquake,NaN,0.70,NaN,NaN,automatic,ak,ak
62579,2022-05-09T04:13:49.615Z,37.202100,-114.879800,5.70,1.60,ml,5.0,231.36,0.24600,0.1384,...,2022-05-09T20:14:59.778Z,"31 km SE of Alamo, Nevada",earthquake,NaN,14.00,0.160000,2.0,reviewed,nn,nn
62580,2022-05-09T04:04:52.770Z,46.805167,-121.920667,7.39,-0.08,md,13.0,156.00,0.04138,0.0800,...,2022-05-09T16:39:04.370Z,"9 km ENE of Ashford, Washington",earthquake,0.32,0.46,0.218354,13.0,reviewed,uw,uw


In [7]:
#dataframe from SQLite
model_df2 = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
model_df2

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.87500,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290000,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.11900,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460000,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62364,2022-05-09T04:32:16.310Z,17.874167,-66.876667,13.74,2.71,md,9.0,245.00,NaN,0.1900,...,2022-05-09T04:45:08.100Z,"11 km SSE of Guánica, Puerto Rico",earthquake,1.04,0.49,0.239170,7.0,reviewed,pr,pr
62365,2022-05-09T04:27:04.998Z,59.922600,-152.162900,79.90,1.50,ml,NaN,NaN,NaN,0.5300,...,2022-05-09T04:30:31.925Z,"24 km W of Happy Valley, Alaska",earthquake,NaN,0.70,NaN,NaN,automatic,ak,ak
62366,2022-05-09T04:13:49.615Z,37.202100,-114.879800,5.70,1.60,ml,5.0,231.36,0.24600,0.1384,...,2022-05-09T20:14:59.778Z,"31 km SE of Alamo, Nevada",earthquake,NaN,14.00,0.160000,2.0,reviewed,nn,nn
62367,2022-05-09T04:04:52.770Z,46.805167,-121.920667,7.39,-0.08,md,13.0,156.00,0.04138,0.0800,...,2022-05-09T16:39:04.370Z,"9 km ENE of Ashford, Washington",earthquake,0.32,0.46,0.218354,13.0,reviewed,uw,uw


In [8]:
#check for duplicates in postgres data
boolean = model_df.duplicated(subset=['id']).any()
print(boolean)

True


In [9]:
#check for duplicates in sqlite data
boolean = model_df2.duplicated(subset=['id']).any()
print(boolean)

True


In [10]:
#drop duplicates in postgres data
model_df.drop_duplicates(subset=['id'], inplace=True)
print(model_df)

                           time   latitude   longitude   depth   mag magType  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.107500    1.10  1.30      ml   
1      2022-01-06T01:31:33.698Z  38.181000 -117.818000    7.30  1.40      ml   
2      2022-01-06T01:24:20.083Z  61.340700 -149.885200   30.00  1.50      ml   
3      2022-01-06T01:15:15.851Z  62.870300 -150.762500  111.30  2.10      ml   
4      2022-01-06T01:10:44.629Z  64.014500 -148.765700    1.90  1.50      ml   
...                         ...        ...         ...     ...   ...     ...   
62075  2022-05-10T18:52:32.159Z -27.668500 -176.934500   25.08  4.60      mb   
62076  2022-05-10T18:49:23.310Z -59.013100  -25.575000   55.95  4.90      mb   
62105  2022-05-10T16:40:16.050Z  34.731833 -119.073333   11.19  0.83      ml   
62208  2022-05-10T08:28:40.129Z   2.945500  128.323100   53.98  4.50      mb   
62313  2022-05-10T00:19:13.902Z  -4.962200  102.541400   41.84  4.30      mb   

        nst     gap     dmin     rms  .

In [11]:
#drop duplicates in sqlite data
model_df2.drop_duplicates(subset=['id'], inplace=True)
print(model_df2)

                           time   latitude   longitude   depth   mag magType  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.107500    1.10  1.30      ml   
1      2022-01-06T01:31:33.698Z  38.181000 -117.818000    7.30  1.40      ml   
2      2022-01-06T01:24:20.083Z  61.340700 -149.885200   30.00  1.50      ml   
3      2022-01-06T01:15:15.851Z  62.870300 -150.762500  111.30  2.10      ml   
4      2022-01-06T01:10:44.629Z  64.014500 -148.765700    1.90  1.50      ml   
...                         ...        ...         ...     ...   ...     ...   
61860  2022-05-10T18:52:32.159Z -27.668500 -176.934500   25.08  4.60      mb   
61861  2022-05-10T18:49:23.310Z -59.013100  -25.575000   55.95  4.90      mb   
61890  2022-05-10T16:40:16.050Z  34.731833 -119.073333   11.19  0.83      ml   
61995  2022-05-10T08:28:40.129Z   2.945500  128.323100   53.98  4.50      mb   
62100  2022-05-10T00:19:13.902Z  -4.962200  102.541400   41.84  4.30      mb   

        nst     gap     dmin     rms  .

In [12]:
#replace the table in postgres with one without duplicates
model_df.to_sql('earthquake_raw', engine, if_exists='replace', index = False)

111

In [13]:
#replace the table in sqlite with one without duplicates
model_df2.to_sql('earthquake_raw', conn, if_exists='replace', index = False)

58108

In [14]:
#copy data as csv as backup method
model_df.to_csv("../Database/csv/earthquake_raw.csv",index=False)

In [63]:
# Import API key
import sys
weather_api_key = "YOUR KEY HERE!"

if weather_api_key == "YOUR KEY HERE!":
    sys.path.append(r"/Users/rosaicelaroman/Desktop/Data_BootCamp/APIs/config_keys/")
    from api_keys_v2 import weather_api_key

In [39]:
#create new dataframe for locatiions
# location_df = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
# location_df

In [40]:
#create new df with only necessary columns
# cleaned_df = location_df.drop(columns=['nst','rms','magNst','net', 'time', 'magType', 'gap','dmin', 'updated', 'place', 'type', 'horizontalError', 'depthError', 'magError', 'status', 'locationSource', 'magSource'])
# cleaned_df

http://api.openweathermap.org/geo/1.0/reverse?lat={lat}&lon={lon}&limit={limit}&appid={API key}

In [41]:
#create new columns for data from API
# cleaned_df['Location Name']=""
# cleaned_df['City']=""
# cleaned_df['State']=""
# cleaned_df['Country']=""
# cleaned_df

In [64]:
location_df = pd.read_sql_query("SELECT * FROM earthquake_raw_locations", conn)
location_df

,latitude,longitude,depth,mag,id,Location Name,City,State,Country
0,37.384000,-117.107500,1.10,1.30,nn00831163,Nye County,Nye County,Nevada,US
1,38.181000,-117.818000,7.30,1.40,nn00831161,Esmeralda County,Esmeralda County,Nevada,US
2,61.340700,-149.885200,30.00,1.50,ak0229xypaw,Matanuska-Susitna,Matanuska-Susitna,Alaska,US
3,62.870300,-150.762500,111.30,2.10,ak0229xwriz,Matanuska-Susitna,Matanuska-Susitna,Alaska,US
4,64.014500,-148.765700,1.90,1.50,ak0229xvr66,Denali,Denali,Alaska,US
...,...,...,...,...,...,...,...,...,...
38781,35.322667,-117.783333,4.70,1.09,ci39983311,Kern County,Kern County,California,US
38782,63.390200,-151.341800,0.00,1.80,ak02246jyds7,Denali,Denali,Alaska,US
38783,60.602200,-150.643000,19.40,2.10,ak02246jxbq8,Kenai Peninsula,Kenai Peninsula,Alaska,US
38784,35.965500,-117.677167,2.37,0.79,ci39983287,Inyo County,Inyo County,California,US


In [65]:
base_url = "http://api.openweathermap.org/geo/1.0/reverse?"



http://api.openweathermap.org/geo/1.0/reverse?lat=51.5098&lon=-0.1180&limit=5&appid={API key}

In [49]:
# response = requests.get(base_url+"lat="+'37.3840'+'&lon='+'-117.107500'+'&limit='+'1'+'&appid=' + weather_api_key).json()
# print(response[0]['country'])

US


In [66]:
import requests
import time

for index, row in location_df.iterrows():
    # count+=1
    lat = row['latitude']
    lon = row['longitude']
    loc_found = []

    if row['Country'] != "":
        continue

    print(f'Retrieving results for Index {index}: {lat}')
    try:
        response = requests.get(base_url+"lat="+str(lat)+'&lon='+str(lon)+'&limit='+'1'+'&appid=' + weather_api_key).json()
        # response = requests.get(base_url+city+"&units="+units+'&appid=' + weather_api_key).json()
        location_df.loc[index, 'Location Name'] = response[0]['name']
        location_df.loc[index, 'City'] = response[0]['local_names']['en']
        location_df.loc[index, 'State'] = response[0]['state']
        location_df.loc[index, 'Country'] = response[0]['country']
        loc_found.append(lat)

        print(f'Found lat: {lat}')
    except:
        print(f"Missing lat: {lat}... Skipping")
        location_df.loc[index, 'Location Name'] = float('NaN')
        location_df.loc[index, 'City'] = float('NaN')
        location_df.loc[index, 'State'] = float('NaN')
        location_df.loc[index, 'Country'] = float('NaN')
    print(f'--------------------')
    time.sleep(2)


In [46]:
location_df

,latitude,longitude,depth,mag,id,Location Name,City,State,Country
0,37.384000,-117.107500,1.10,1.30,nn00831163,Nye County,Nye County,Nevada,US
1,38.181000,-117.818000,7.30,1.40,nn00831161,Esmeralda County,Esmeralda County,Nevada,US
2,61.340700,-149.885200,30.00,1.50,ak0229xypaw,Matanuska-Susitna,Matanuska-Susitna,Alaska,US
3,62.870300,-150.762500,111.30,2.10,ak0229xwriz,Matanuska-Susitna,Matanuska-Susitna,Alaska,US
4,64.014500,-148.765700,1.90,1.50,ak0229xvr66,Denali,Denali,Alaska,US
...,...,...,...,...,...,...,...,...,...
38781,35.322667,-117.783333,4.70,1.09,ci39983311,Kern County,Kern County,California,US
38782,63.390200,-151.341800,0.00,1.80,ak02246jyds7,Denali,Denali,Alaska,US
38783,60.602200,-150.643000,19.40,2.10,ak02246jxbq8,Kenai Peninsula,Kenai Peninsula,Alaska,US
38784,35.965500,-117.677167,2.37,0.79,ci39983287,Inyo County,Inyo County,California,US


In [47]:
#replace the table in sqlite with one without duplicates
location_df.to_sql('earthquake_raw_locations', conn, if_exists='replace', index = False)

#replace the table in postgres with one without duplicates
location_df.to_sql('earthquake_raw_locations', engine, if_exists='replace', index = False)

786

In [48]:
# Close connection to SQLite database
conn.close()
connection.close()